# 01. ChEMBL Preprocessing

In [1]:
from pathlib import Path

import numpy as np
import pandas as pd
import sqlite3

Download 34th version of ChEMBL SQLite database from here https://ftp.ebi.ac.uk/pub/databases/chembl/ChEMBLdb/latest/ and unpack it:

In [ ]:
!mkdir -p ../data
!wget https://ftp.ebi.ac.uk/pub/databases/chembl/ChEMBLdb/latest/chembl_34_sqlite.tar.gz -O ../data/chembl_34_sqlite.tar.gz
!tar -xzf ../data/chembl_34_sqlite.tar.gz -C ../data

--2024-11-12 15:33:01--  https://ftp.ebi.ac.uk/pub/databases/chembl/ChEMBLdb/latest/chembl_34_sqlite.tar.gz
Resolving ftp.ebi.ac.uk (ftp.ebi.ac.uk)... 193.62.193.165
Connecting to ftp.ebi.ac.uk (ftp.ebi.ac.uk)|193.62.193.165|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4859330198 (4.5G) [application/x-gzip]
Saving to: ‘../data/chembl_34_sqlite.tar.gz’

../data/chembl_34_s 100%[===================>]   4.53G  30.8MB/s    in 2m 36s  

2024-11-12 15:35:38 (29.7 MB/s) - ‘../data/chembl_34_sqlite.tar.gz’ saved [4859330198/4859330198]



In [3]:
DATA_DIR = Path('../data')
PATH_TO_CHEMBL = DATA_DIR / 'chembl_34/chembl_34_sqlite/chembl_34.db'

In [4]:
connection = sqlite3.connect(PATH_TO_CHEMBL)
chembl = connection.cursor()

In [5]:
request = '''
SELECT
    cs.canonical_smiles,
    cmpd.accession AS protein_accession,
    act.standard_type,
    act.standard_value,
    act.standard_units,
    act.standard_relation,
    td.organism,
    td.pref_name AS target_name,
    md.chembl_id AS compound_chembl_id,
    td.chembl_id AS target_chembl_id,
    a.assay_id,
    cmpd.sequence AS protein_sequence
FROM
    activities act
    JOIN assays a ON act.assay_id = a.assay_id
    JOIN target_dictionary td ON a.tid = td.tid
    JOIN target_components tc ON td.tid = tc.tid
    JOIN component_sequences cmpd ON tc.component_id = cmpd.component_id
    JOIN organism_class oc ON td.tax_id = oc.tax_id
    JOIN molecule_dictionary md ON act.molregno = md.molregno
    JOIN compound_structures cs ON md.molregno = cs.molregno
    JOIN docs d ON a.doc_id = d.doc_id
WHERE
    a.confidence_score = 9
    AND td.target_type = 'SINGLE PROTEIN'
    AND oc.L2 = 'Mammalia'
    AND act.standard_type IN ('Ki', 'Kd', 'IC50')
    AND act.standard_value IS NOT NULL
    AND act.standard_units IN ('nM')
    AND act.standard_flag = 1
    AND act.potential_duplicate = 0
    AND act.data_validity_comment IS NULL
    AND act.activity_comment IS NULL
    AND a.assay_type IN ('B', 'F')
    AND act.src_id NOT IN (SELECT src_id FROM source WHERE src_description LIKE '%unreliable_source%')
ORDER BY
    td.pref_name, act.standard_value;
'''

In [6]:
result = chembl.execute(request).fetchall()

In [7]:
data = pd.DataFrame(result, columns=[
    'smiles',
    'uniprot_id',
    'type',
    'value',
    'units',
    'relation',
    'organism',
    'target_name',
    'compound_chembl_id',
    'target_chembl_id',
    'assay_id',
    'sequence'
])
data

,smiles,uniprot_id,type,value,units,relation,organism,target_name,compound_chembl_id,target_chembl_id,assay_id,sequence
0,CCc1c(-c2nc(N)nc(Nc3ccc(C)cc3)n2)oc2ccccc12,O15120,IC50,8.4,nM,=,Homo sapiens,1-acylglycerol-3-phosphate O-acyltransferase beta,CHEMBL5219455,CHEMBL4772,2261572,MELWPCLAAALLLLLLLVQLSRAAEFYAKVALYCALCFTVSAVASL...
1,CCc1ccc(Nc2nc(N)nc(-c3oc4ccccc4c3C)n2)cc1,O15120,IC50,15.0,nM,=,Homo sapiens,1-acylglycerol-3-phosphate O-acyltransferase beta,CHEMBL5219496,CHEMBL4772,2261572,MELWPCLAAALLLLLLLVQLSRAAEFYAKVALYCALCFTVSAVASL...
2,COc1ccc(Cl)cc1-c1cc(N)nc(Nc2ccc(Cl)cc2)c1,O15120,IC50,40.0,nM,=,Homo sapiens,1-acylglycerol-3-phosphate O-acyltransferase beta,CHEMBL193874,CHEMBL4772,321473,MELWPCLAAALLLLLLLVQLSRAAEFYAKVALYCALCFTVSAVASL...
3,COc1ccc(Cl)cc1-c1nc(N)nc(Nc2ccc(Cl)cc2)n1,O15120,IC50,45.0,nM,=,Homo sapiens,1-acylglycerol-3-phosphate O-acyltransferase beta,CHEMBL195457,CHEMBL4772,321473,MELWPCLAAALLLLLLLVQLSRAAEFYAKVALYCALCFTVSAVASL...
4,CCOc1ccc(Cl)cc1-c1cc(N)nc(Nc2ccc(Cl)cc2)c1,O15120,IC50,50.0,nM,=,Homo sapiens,1-acylglycerol-3-phosphate O-acyltransferase beta,CHEMBL194604,CHEMBL4772,321473,MELWPCLAAALLLLLLLVQLSRAAEFYAKVALYCALCFTVSAVASL...
...,...,...,...,...,...,...,...,...,...,...,...,...
917892,Nc1ncnc2c1ncn2[C@@H]1O[C@H](CN(CC[C@H](N)C(=O)...,O14717,Kd,45100.0,nM,=,Homo sapiens,tRNA (cytosine(38)-C(5))-methyltransferase,CHEMBL5277756,CHEMBL4523124,2292005,MEPLRVLELYSGVGGMHHALRESCIPAQVVAAIDVNTVANEVYKYN...
917893,COc1ccc(CN(CC[C@H](N)C(=O)O)C[C@H]2O[C@@H](n3c...,O14717,Kd,55800.0,nM,=,Homo sapiens,tRNA (cytosine(38)-C(5))-methyltransferase,CHEMBL5267824,CHEMBL4523124,2292005,MEPLRVLELYSGVGGMHHALRESCIPAQVVAAIDVNTVANEVYKYN...
917894,Nc1ncnc2c1ncn2[C@@H]1O[C@H](CN(CC[C@H](N)C(=O)...,O14717,IC50,62800.0,nM,=,Homo sapiens,tRNA (cytosine(38)-C(5))-methyltransferase,CHEMBL5272276,CHEMBL4523124,2292001,MEPLRVLELYSGVGGMHHALRESCIPAQVVAAIDVNTVANEVYKYN...
917895,C#CCN(CC[C@H](N)C(=O)O)C[C@H]1O[C@@H](n2cnc3c(...,O14717,IC50,77100.0,nM,=,Homo sapiens,tRNA (cytosine(38)-C(5))-methyltransferase,CHEMBL1091913,CHEMBL4523124,2223596,MEPLRVLELYSGVGGMHHALRESCIPAQVVAAIDVNTVANEVYKYN...


In [8]:
data['units'].value_counts()

units
nM    917897
Name: count, dtype: int64

For the further filtration we need less columns.

In [9]:
data = data[[
    'smiles',
    'type',
    'value',
    'units',
    'relation',
    'sequence',
    'uniprot_id',
]]
data

,smiles,type,value,units,relation,sequence,uniprot_id
0,CCc1c(-c2nc(N)nc(Nc3ccc(C)cc3)n2)oc2ccccc12,IC50,8.4,nM,=,MELWPCLAAALLLLLLLVQLSRAAEFYAKVALYCALCFTVSAVASL...,O15120
1,CCc1ccc(Nc2nc(N)nc(-c3oc4ccccc4c3C)n2)cc1,IC50,15.0,nM,=,MELWPCLAAALLLLLLLVQLSRAAEFYAKVALYCALCFTVSAVASL...,O15120
2,COc1ccc(Cl)cc1-c1cc(N)nc(Nc2ccc(Cl)cc2)c1,IC50,40.0,nM,=,MELWPCLAAALLLLLLLVQLSRAAEFYAKVALYCALCFTVSAVASL...,O15120
3,COc1ccc(Cl)cc1-c1nc(N)nc(Nc2ccc(Cl)cc2)n1,IC50,45.0,nM,=,MELWPCLAAALLLLLLLVQLSRAAEFYAKVALYCALCFTVSAVASL...,O15120
4,CCOc1ccc(Cl)cc1-c1cc(N)nc(Nc2ccc(Cl)cc2)c1,IC50,50.0,nM,=,MELWPCLAAALLLLLLLVQLSRAAEFYAKVALYCALCFTVSAVASL...,O15120
...,...,...,...,...,...,...,...
917892,Nc1ncnc2c1ncn2[C@@H]1O[C@H](CN(CC[C@H](N)C(=O)...,Kd,45100.0,nM,=,MEPLRVLELYSGVGGMHHALRESCIPAQVVAAIDVNTVANEVYKYN...,O14717
917893,COc1ccc(CN(CC[C@H](N)C(=O)O)C[C@H]2O[C@@H](n3c...,Kd,55800.0,nM,=,MEPLRVLELYSGVGGMHHALRESCIPAQVVAAIDVNTVANEVYKYN...,O14717
917894,Nc1ncnc2c1ncn2[C@@H]1O[C@H](CN(CC[C@H](N)C(=O)...,IC50,62800.0,nM,=,MEPLRVLELYSGVGGMHHALRESCIPAQVVAAIDVNTVANEVYKYN...,O14717
917895,C#CCN(CC[C@H](N)C(=O)O)C[C@H]1O[C@@H](n2cnc3c(...,IC50,77100.0,nM,=,MEPLRVLELYSGVGGMHHALRESCIPAQVVAAIDVNTVANEVYKYN...,O14717


In [10]:
rows_to_drop = data['relation'].isin(['~', '>>'])
data = data[~rows_to_drop]

Create columns with value and sign for each data type

In [11]:
def get_activities(row):
    relation = row['relation']
    # Standardize the relation between datasets
    if relation == '>=':
        relation = '>'
    if relation == '<=':
        relation = '<'
    
    # If value or relation is missing, return NaN
    ki = np.nan
    ki_sign = None
    kd = np.nan
    kd_sign = None
    ic50 = np.nan
    ic50_sign = None
    ec50 = np.nan
    ec50_sign = None

    if row['type'] == 'Ki':
        ki = row['value']
        ki_sign = relation
    elif row['type'] == 'Kd':
        kd = row['value']
        kd_sign = relation
    elif row['type'] == 'IC50':
        ic50 = row['value']
        ic50_sign = relation
    elif row['type'] == 'EC50':
        ec50 = row['value']
        ec50_sign = relation
    
    return ki, kd, ic50, ec50, ki_sign, kd_sign, ic50_sign, ec50_sign

In [12]:
activities = pd.DataFrame(
    data.apply(get_activities, axis=1).to_list(),
    columns=['ki', 'kd', 'ic50', 'ec50', 'ki_sign', 'kd_sign', 'ic50_sign', 'ec50_sign']
)

data = pd.concat([
    data[['smiles', 'sequence', 'uniprot_id']], 
    activities
], axis=1)
data['source'] = 'chembl'
data

,smiles,sequence,uniprot_id,ki,kd,ic50,ec50,ki_sign,kd_sign,ic50_sign,ec50_sign,source
0,CCc1c(-c2nc(N)nc(Nc3ccc(C)cc3)n2)oc2ccccc12,MELWPCLAAALLLLLLLVQLSRAAEFYAKVALYCALCFTVSAVASL...,O15120,NaN,NaN,8.4,NaN,None,None,=,None,chembl
1,CCc1ccc(Nc2nc(N)nc(-c3oc4ccccc4c3C)n2)cc1,MELWPCLAAALLLLLLLVQLSRAAEFYAKVALYCALCFTVSAVASL...,O15120,NaN,NaN,15.0,NaN,None,None,=,None,chembl
2,COc1ccc(Cl)cc1-c1cc(N)nc(Nc2ccc(Cl)cc2)c1,MELWPCLAAALLLLLLLVQLSRAAEFYAKVALYCALCFTVSAVASL...,O15120,NaN,NaN,40.0,NaN,None,None,=,None,chembl
3,COc1ccc(Cl)cc1-c1nc(N)nc(Nc2ccc(Cl)cc2)n1,MELWPCLAAALLLLLLLVQLSRAAEFYAKVALYCALCFTVSAVASL...,O15120,NaN,NaN,45.0,NaN,None,None,=,None,chembl
4,CCOc1ccc(Cl)cc1-c1cc(N)nc(Nc2ccc(Cl)cc2)c1,MELWPCLAAALLLLLLLVQLSRAAEFYAKVALYCALCFTVSAVASL...,O15120,NaN,NaN,50.0,NaN,None,None,=,None,chembl
...,...,...,...,...,...,...,...,...,...,...,...,...
848574,NaN,NaN,NaN,NaN,9310.0,NaN,NaN,None,=,None,None,chembl
848575,NaN,NaN,NaN,NaN,9800.0,NaN,NaN,None,=,None,None,chembl
848576,NaN,NaN,NaN,NaN,10000.0,NaN,NaN,None,>,None,None,chembl
902552,NaN,NaN,NaN,169.82,NaN,NaN,NaN,=,None,None,None,chembl


In [13]:
data.to_csv(DATA_DIR / 'chembl.csv', index=False)